1. 파일 불러오기
2. 텍스트 파일 전처리
3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기
4. keyword extraction 모델 적용
5. 텍스트 단어 대체
######  https://lovit.github.io/nlp/2018/04/16/krwordrank/

#1. 파일 구글 드라이브로 가져와서 불러오기
- 두 가지 방법 중 하나만 선택

###### 직접 upload

In [11]:
#내 드라이브에 있는 파일 google drive에 업로드
from google.colab import files
myfile = files.upload()

KeyboardInterrupt: ignored

In [10]:
#파일 불러오기
import io
import os
import pandas as pd
import numpy as np

texts = open("정보처리산업기사_2과목요점정리.txt","r", encoding='utf-8')

TypeError: ignored

##### google drive mount

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
filename = '/content/drive/My Drive/OpenSource/data/정보처리산업기사_1과목요점정리.txt'

In [46]:
with open(filename,"r", encoding='utf-8') as text_file:
    texts = text_file.read()

lines = texts.split("\n") #texts를 개행 문자 기준으로 split하여 list로 저장 

for i in range(len(lines)):
    lines[i] = lines[i].replace("\t", " ") #각 문장에서 \t를 띄어쓰기로 대체

#2. 텍스트 파일 전처리
- 네이버 한글 맞춤법 검사기를 바탕으로 맞춤법, 띄어쓰기 올바르게 교정

###### https://wikidocs.net/92961

In [8]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-xwlw4tvx
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-xwlw4tvx
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=3bbd4d4db4d656aa2f9bd1a31931cc3a79db8491954a295329c0471202eb464f
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9bie4mf/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [48]:
#각 줄 별로 맞춤법, 띄어쓰기 교정
for i in range(len(lines)):
    lines[i] = spell_checker.check(lines[i])
    lines[i]  = lines[i].checked
    print(lines[i])

정보시스템 
▪조직체에 필요한 DATA를 수집, 저장해 두었다가 필요시에 처리해서 의사결정에 유용한 정보를 생성하고 분배하는 수단
▪사용하는 목적에 따라 경영정보 시스템, 군사 정보시스템, 인사 행정 정보 시스템, 의사 결정 지원 시스템 등으로 사용됨
정보와 자료
▪자료(Data) : 현실세계에서 관찰이나 측정을 통해 수집한 단순한 사실이나 결괏값으로, 가공되지 않은 상태
▪정보(Information) : 의사 결정에 도움을 줄 수 있는 유용한 형태로, 자료를 가공(처리) 해서 얻을 수 있는 결과
                   자료 처리 시스템
   자료  →                              →   정보

▪자료처리 시스템 : 정보시스템이 사용할 자료를 처리하는 정보 시스템의 서브시스템으로, 처리 형태에 따라 일괄 처리 시스템, 온라인 실시간 처리 시스템, 분산 처리 시스템으로 분류됨
▪데이터 웨어 하우스(DataWare House) : 조직이나 기업체의 중심이 되는 주요 업무 시스템에서 추출되어 새로이 생성된 데이터베이스로서 의사결정지원 시스템을 지원하는 주체적, 통합적, 시간적 데이터의 집합체


▪통합된 데이터(Integrated data) : 자료의 중복을 배제한 데이터의 모임
▪저장된 데이터(Stored data) : 컴퓨터가 접근할 수 있는 저장 매체에 저장된 자료
▪운영 데이터(Operational data) : 조직의 업무를 수행하는 데 있어서 존재 가치가 확실하고 없어서는 안 될 반드시 필요한 자료
▪공용 데이터 : 여러 응용 시스템들이 공동으로 소유하고 유지하는 자료


▪실시간 접근성 : 수시적이고 비정형적인 질의(조회)에 대하여 실시간 처리(real time processing) 응답이 가능함
▪계속적인 변화 : 새로운 데이터의 삽입(insertion), 삭제(deletion), 갱신(update)으로 항상 최신의 데이터를 유지함
▪동시 공용 : 여러 사용자가 동시에 자기가 원하는 데이터를 이용할 수 있음
▪내용에 

# 3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기

##### 단어의 최소 출현 빈도수와 단어의 최대 길이를 결정짓기 위함 

In [49]:
#문장속 명사만 추출
#konlpy의 tag
!pip install konlpy
from konlpy.tag import Twitter
from collections import Counter

In [52]:
nlpy = Twitter()

#lines = texts.read()
#nouns = nlpy.nouns(lines)#명사만 추출
for i in lines:
    nouns = nlpy.nouns(i)

print(len(nouns))#추출된 명사의 갯수

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


AttributeError: ignored

In [37]:
# 명사의 빈도수 계산
from collections import Counter
count = Counter(nouns)

tag_count = []
tags = []

#길이가 2부터 49까지인 단어 
for n, c in count.most_common():
  dics = {'tag': n, 'count': c}
  if len(dics['tag']) >= 2 and len(tags) <= 49:
    tag_count.append(dics)
    tags.append(dics['tag'])

In [38]:
for tag in tag_count:
  print(" {:<14}".format(tag['tag']), end='\t')
  print("{}".format(tag['count']))

 장치            	155
 전송            	114
 메모리           	94
 사용            	86
 명령어           	84
 출력            	80
 데이터           	76
 인터럽트          	76
 명령            	74
 기억            	71
 레지스터          	68
 방식            	67
 코드            	67
 주소            	63
 프로그램          	60
 연산            	60
 비트            	59
 제어            	59
 자료            	57
 보수            	49
 번지            	49
 신호            	48
 처리            	45
 단계            	45
 정보            	43
 구성            	42
 발생            	42
 실행            	41
 내용            	41
 표현            	40
 시간            	40
 논리            	38
 위치            	38
 레코드           	36
 수행            	36
 부분            	35
 저장            	35
 지정            	34
 이용            	34
 주기억장치         	34
 변천            	34
 속도            	33
 접근            	30
 진수            	29
 동작            	29
 자리            	28
 의미            	26
 버스            	26
 우선            	26
 상태            	25


In [39]:
noun_count_df = pd.DataFrame(tag_count)
noun_count_df

,tag,count
0,장치,155
1,전송,114
2,메모리,94
3,사용,86
4,명령어,84
5,출력,80
6,데이터,76
7,인터럽트,76
8,명령,74
9,기억,71


In [40]:
#명사 중 가장 긴 단어의 문자열 길이 
print("최장 단어 길이 ", max(noun_count_df["tag"].str.len()))
#명사 중 최소 출현 빈도 수
print("최소 출현 빈도 수 ", min(noun_count_df["count"]))

최장 단어 길이  5
최소 출현 빈도 수  25


##### 불필요한 문자 제거 (영어, 숫자 옵션 선택 가능) 

In [41]:
!pip install krwordrank
from krwordrank.hangle import normalize

#normalize는 불필요한 특수 기호를 제거
#english=True, number =True를 입력하면 한글, 영어, 숫자를 제외한 다른 글자들을 제거함.
#default는 english =False, number= False
lines = [normalize(lines, english=True, number=True)]

In [42]:
lines

['전자 계산기 구조 교환법칙 A B B A A B B A 결합법칙 A B C A B C A B C A B C 분배법칙 A B C A B A C A B C A B A C 멱등법칙 A A A A A A 보수법칙 A A 1 A A 0 항등법칙 A 0 A A 1 1 A 0 0 A 1 A 콘센서스 AB BC CA AB CA A B B C C A A B C A 드모르강 A B A B A B A B 복원법칙 A A 게이트 기호 의미 진리표 논리식 AND 입력신호가 모두 1일 때 1 출력 A B Y 0 0 0 0 1 0 1 0 0 1 1 1 Y A B Y AB OR 입력신호 중 1개만 1이어도 1 출력 A B Y 0 0 0 0 1 1 1 0 1 1 1 1 Y A B NOT 입력된 정보를 반대로 변환하여 출력 A Y 0 1 1 0 Y A Y A BUFFER 입력된 정보를 그대로 출력 A Y 0 0 1 1 Y A NAND NOT AND 즉 AND의 부정 A B Y 0 0 1 0 1 1 1 0 1 1 1 0 Y A B NOR NOT OR 즉 OR의 부정 A B Y 0 0 1 0 1 0 1 0 0 1 1 0 Y A B XOR 입력되는 값이 모두 같은면 0 한개라도 틀리면 1출력 A B Y 0 0 0 0 1 1 1 0 1 1 1 0 Y A B Y A B AB Y A B A B Y A B AB XNOR NOT XOR 즉 XOR의 부정 A B Y 0 0 1 0 1 0 1 0 0 1 1 1 Y A B Y X Y Y AB A B Y A B A B Y AB A B 1Bit짜리 2진수 2개를 덧셈한 합 S 과 자리올림 수 C 를 구하는 회로 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 A B C S S 합 C 자리 올림 진리표 A B S C 0 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 논리식 C A B S A B A B A B 논리회로 자리 올림수 Ci 를 포함하여 1Bit 크기의 2진수 3자리를 더하여 합 Si 과 자리 올림수 Ci 1 를 구하는 회로 논리식 Ci 

#Keyword

In [44]:
from krwordrank.word import KRWordRank

wordrank_extractor = KRWordRank(
    min_count = min(noun_count_df["count"]), # 단어의 최소 출현 빈도수
    max_length = max(noun_count_df["tag"].str.len()), # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

#keywords는 filtering이 적용된 L parts
#rank는 substriing graph의 substring에 대한 구마
#graph는 substring graph
keywords, rank, graph = wordrank_extractor.extract(lines, beta, max_iter)

scan vocabs ... 
num vocabs = 326
done


In [45]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:100]:
    print('%8s:\t%.4f' % (word, r))

      있는:	3.2142
      명령:	2.9664
    인터럽트:	2.3723
     전송함:	2.2387
      출력:	2.1644
     메모리:	2.0960
      있음:	2.0617
     MBR:	1.9494
    레지스터:	1.8659
Operation:	1.8536
     단계로:	1.8297
    데이터를:	1.7574
      번지:	1.7033
     CPU:	1.6945
      코드:	1.6765
    Time:	1.6632
      장치:	1.6247
    제어신호:	1.5718
      사용:	1.5389
    프로그램:	1.5163
     변천함:	1.5048
      주소:	1.4839
      값을:	1.4567
     MAR:	1.4471
      방식:	1.4470
      기억:	1.3868
   Micro:	1.3762
      논리:	1.3559
      비트:	1.1767
      PC:	1.1718
      AC:	1.1634
Register:	1.1589
      또는:	1.1322
      모든:	1.1246
      연산:	1.1185
      처리:	1.1041
Interrupt:	1.0938
      의미:	1.0488
      1을:	1.0479
     정보를:	1.0317
      자료:	1.0148
     레코드:	1.0130
     C2t:	0.9964
      보수:	0.9295
      구성:	0.9161
      실행:	0.8640
      상태:	0.8412
      단위:	0.8062
      여러:	0.7966
      표현:	0.7916
      발생:	0.7858
      내용:	0.7674
      저장:	0.7313
     Bit:	0.7294
      접근:	0.7024
      문자:	0.7010
      시간:	0.6861
      지정:	0.6597
      위치:	0.

# 텍스트에서 위의 주요 단어 빈칸으로 대체

In [46]:
keywords_list = keywords.keys()
keywords_list

dict_keys(['있는', '명령', '인터럽트', '전송함', '출력', '메모리', '있음', 'MBR', '레지스터', 'Operation', '단계로', '데이터를', '번지', 'CPU', '코드', 'Time', '장치', '제어신호', '사용', '프로그램', '변천함', '주소', '값을', 'MAR', '방식', '기억', 'Micro', '논리', '비트', 'PC', 'AC', 'Register', '또는', '모든', '연산', '처리', 'Interrupt', '의미', '1을', '정보를', '자료', '레코드', 'C2t', '보수', '구성', '실행', '상태', '단위', '여러', '표현', '발생', '내용', '저장', 'Bit', '접근', '문자', '시간', '지정', '위치', '부분', '동작', '수행', '이용', 'Co', '10', '2진', '주기억', '우선', 'AB', '1B', '자기', 'St'])

In [48]:
# 입력받은 txt 파일 읽기 모드로 재호출
f = open("정보처리산업기사_2과목요점정리.txt","r", encoding='utf-8')

lines = []
for paragraph in f:
  lines = string.split(paragraph, ["\n","."])
  for each_line in lines:
    if any(each_line in word for word in keywords_list):
      print("O")

NameError: ignored

In [ ]:
lines = f.readlines()
lines[:]